In [1]:
def pxlocal(line, cell):
    ip = get_ipython()
    ip.run_cell_magic("px", line, cell)
    ip.run_cell(cell)
get_ipython().register_magic_function(pxlocal, "cell")

In [2]:
import subprocess
import ipyparallel as ipp
import time
from IPython.display import clear_output

subprocess.Popen(["ipcluster", "stop"])
time.sleep(10)

num_agents = 2
num_engines = 2
subprocess.Popen(["ipcluster", "start", "-n={:d}".format(num_engines)])

wait_time = 40
# Waiting for clusters to start properly
for i in range(wait_time):
    clear_output(wait = True)
    print('Waiting for', wait_time - (i+1), 'seconds')
    time.sleep(1)
    
rc = ipp.Client()

Waiting for 0 seconds


In [3]:
%%pxlocal

from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym import wrappers
from IPython.display import Video
import io
import base64
from IPython.display import HTML
import numpy as np

class Agent:
    
    def __init__(self):
        self.state = None
        self.reward = None
        self.done = None
        self.info = None
        self.next_state = None
        
    def take_action(self, env, mode = None):
        return env.action_space.sample()
    
    def make_env(self, mode = None):
        env = gym_super_mario_bros.make('SuperMarioBros-v0')
        env = JoypadSpace(env, SIMPLE_MOVEMENT)
        if mode == 'monitor':
            env = wrappers.Monitor(env, directory, force = True)
        return env
    
    def reset_data(self):
        self.state = []
        self.reward = []
        self.done = []
        self.info = dict()
        self.next_state = []
    
    def update(self, state, reward, done, info, next_state):
        self.state.append(np.array(state))
        self.reward.append(reward)
        self.done.append(done)
        for key, value in info.items():
            if key not in self.info:
                self.info[key] = []
            self.info[key].append(value)
        self.next_state.append(np.array(next_state))
        
    def run(self, max_steps = 500, mode = None, directory = './gym-results/'):    
        env = self.make_env(mode = mode)
        self.reset_data()
        
        state = env.reset()
        done = False
        for step in range(max_steps):
            if not done:
                action = self.take_action(env)
                next_state, reward, done, info = env.step(action)
                self.update(state, reward, done, info, next_state)

                if mode == 'render':
                    env.render()

        if mode == 'monitor':
            file_name = directory + 'openaigym.video.%s.video000000.mp4'% env.file_infix
            mp4 = Video(file_name, width = 600, height = 450)
            display(mp4)

        if mode == 'render':    
            env.close()
    
    def get_reward(self):
        if self.reward == None:
            self.run()
        return self.reward
    
    def itsame(self):
        return 'Mario!'

In [13]:
rc = ipp.Client()
generation = [Agent() for i in range(num_agents)]

In [14]:
def parallel_run(rc, generation):
    dview = rc[:]
    ans = dview.map_async(lambda agent: sum(agent.get_reward()), generation).get()
    return ans

In [19]:
start_time = time.time()
ans = parallel_run(rc, generation)
end_time = time.time()

print('Parallel time:', round(end_time - start_time, 3))
print('Ans:', ans)

Parallel time: 1.744
Ans: [529, 369]


In [16]:
start_time = time.time()

ans = [sum(agent.get_reward()) for agent in generation]

end_time = time.time()

print('Sequential time:', round(end_time - start_time, 3))
print('Ans:', ans)

D:\Anaconda3\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


Sequential time: 3.047
Ans: [529, 369]
